<table>
<tr style="border:none;">
<td style="border:none;"><a href="https://fra.utn.edu.ar/">
<image src="https://raw.githubusercontent.com/epplugins/UTN/main/assets/img/logoUTN-500.svg"  width="100">
</a></td>
<td style="border:none;">Física 2<br>2024<br>Electromagnetismo<br>Material para trabajo en clase
</td>
</tr>
</table> 

# Campo eléctrico de distribuciones de cargas contínuas

Parte 1

En este cuaderno los campos se calculan con métodos numéricos simples, con el objetivo de visualizar y comprender como se obtiene el campo total pensando a los objetos extensos como una distribución de muchas cargas puntuales pequeñas y sumando sus contribuciones.

<table>
<tr style="border:none;">
<td style="border:none;"><a href="https://creativecommons.org/licenses/by-nc-sa/4.0/deed.es"><image src="https://raw.githubusercontent.com/epplugins/UTN/main/assets/img/by-nc-sa.svg" width="70" /></a></td>
<td style="border:none;">Edgardo Palazzo (epalazzo@fra.utn.edu.ar)
</td>
</tr>
</table>

---

**Tabla de contenidos**<a id='toc0_'></a>    
- 1. [Campo de una línea recta de carga uniforme](#toc1_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sys
import importlib  # Borrar Not needed after final version.

# ------------------------------------
# Lo siguiente es necesario para que funcione en Google Colab
try:
  import libem as em
except:
  # No descarga los archivos (aunque sí descarga .git)
  !git clone -n https://github.com/epplugins/UTN.git --depth 1
  !cd UTN;git checkout HEAD electromagnetismo/libem.py
  sys.path.insert(0, 'UTN/electromagnetismo')
  import libem as em

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  !pip --quiet install ipympl
  from google.colab import output
  output.enable_custom_widget_manager()
# Fin Google colab
# ------------------------------------

%matplotlib widget

## 1. <a id='toc1_'></a>[Campo de una línea de carga uniforme](#toc0_)

El campo eléctrico producido por una carga puntual $q_0$ ubicada en la posición $\vec{\text{x}}_0$ puede escribirse como:
$$
\vec{E}(\vec{\text{x}}) = \frac{1}{4\pi\varepsilon_0} \frac{q}{|\vec{\text{r}}|^2} \hat{\text{r}}\, , \tag{1}
$$
donde
  * $|\vec{\text{r}}|$ es la distancia entre la posición de la partícula cargada ($\vec{\text{x}}_0$) y la posición donde se quiere obtener el vector de campo eléctrico ($\vec{\text{x}}$)
  * $\hat{\text{r}}$ es el vector unitario en la dirección desde $\vec{\text{x}}_0$ hacia $\vec{\text{x}}$

según se muestra en la siguiente figura.

<div>
       <a href="https://fra.utn.edu.ar/">
       <image src="https://raw.githubusercontent.com/epplugins/UTN/main/electromagnetismo/img/em01_una_carga.svg">
       </a>
</div>

Si el vector unitario se reescribe como $\hat{r} = \vec{\text{r}}/|\vec{\text{r}}|$, la expresión para el campo resulta:
$$
\vec{E}(\vec{\text{x}}) = \frac{1}{4\pi\varepsilon_0} \frac{q}{|\vec{\text{r}}|^3} \vec{\text{r}}\, . \tag{2}
$$

Para escribir un código que calcule el campo eléctrico es conveniente utilizar una expresión con las coordenadas del punto campo y la posición de la carga explícitas. El código que utilizaremos reproduce la siguiente expresión:
$$
\vec{E}(x,y,z) = \frac{1}{4\pi\varepsilon_0} \frac{q}{\left ((x-x_0)^2 + (y-y_0)^2 + (z-z_0)^2 \right )^{3/2}} \left ( (x-x_0)\hat{i} + (y-y_0)\hat{j} + (z-z_0)\hat{k} \right ) . \tag{3}
$$

Definición de una función que calcula las componentes del campo eléctrico en la posición $(x,y,z)$ cuando se tiene una carga de $1~\text{nC}$ en el origen: